In [11]:
import pandas as pd
import numpy as np
from ast import literal_eval


In [12]:
s_list = ["s1", "s2", "s3", "s4"]
data = {}

for s in s_list:
    filename = ["GA1/GA1_{}".format(s), "GA2/GA2_{}".format(s),
                "GA3/GA3_{}".format(s), "RS/RS_{}".format(s)]
    file_out = ["GA1", "GA2", "GA3", "RS"]

    history_last_list = []
    tscc_list = []
    tscc_last_list = []
    tscc_means = []

    for GA in filename:
        tscc = pd.read_csv(GA+"_tscc.csv", index_col="Unnamed: 0")
        tscc_last = tscc.iloc[-1,:].values.tolist()
        tscc_list.append(tscc)
        tscc_last_list.append(tscc_last)
        tscc_means.append(tscc['Mean'].values.tolist()[-1])

        history = pd.read_csv(GA+"_history.csv", index_col="Unnamed: 0")
        history_last = history.iloc[-1,:].values.tolist()
        history_last = [literal_eval(x)['CHROM'] for x in history_last]
        history_last_list.append(history_last)
        last_levels = []

    data[s] = {'hll': history_last_list,
               'tl': tscc_list,
               'tll': tscc_last_list,
               'tmeans': tscc_means}

In [13]:
dict2 = {}
for s in s_list:
    #print(data[s]['hll'])
    tscc_min_index = [np.argmin(x) for x in data[s]['tll']]
    tscc_mins = [np.min(x) for x in data[s]['tll']]
    level_mins = list(map(list, zip(*[x[i] for x, i in zip(data[s]['hll'], tscc_min_index)])))
    dict2[s] = {'R': level_mins[0],
                'D': level_mins[1],
                'M': level_mins[2],
                'S': level_mins[3],
                'tmins': tscc_mins,
                'tmeans': data[s]['tmeans']}

df = pd.concat([pd.DataFrame(dict2['s1']), pd.DataFrame(dict2['s2']),
                pd.DataFrame(dict2['s3']), pd.DataFrame(dict2['s4'])])

#df.to_csv("Data.csv")

In [16]:
dict3 = {}
for s in s_list:
    tscc_last_sd = [np.std(x) for x in data[s]['tll']]
    levels_last_sd = [[np.std([x[0] for x in y]), np.std([x[1] for x in y]), np.std([x[2] for x in y]), np.std([x[3] for x in y])] for y in data[s]['hll']]
    dict3[s] = {'R': levels_last_sd[0],
                'D': levels_last_sd[1],
                'M': levels_last_sd[2],
                'S': levels_last_sd[3],
                'tscc_sd': tscc_last_sd}

df2 = pd.concat([pd.DataFrame(dict3['s1']), pd.DataFrame(dict3['s2']),
                pd.DataFrame(dict3['s3']), pd.DataFrame(dict3['s4'])])

df2 = np.round(df2, 4)
#df2.to_csv("SD_Data.csv")